 # Multiple Simulations on a Server

In this notebook we are going to perform multiple simulations and setting default visualisations for each on a single NanoVer OmniRunner server. <br>
Once they are loaded, the user can run the first one and then pass to the second, third (etc.), via commands in the Jupyter notebook or the VR interface. <br>
Firstly, import the classes from the NanoVer package:

In [1]:
from nanover.omni import OmniRunner
from nanover.openmm import OpenMMSimulation
from nanover.app import RenderingSelection

Then load the simulation files, we are loading three different protein-ligand complexes:
1. Neuraminidase and zanamivir;
2. HIV1 and Amprenavir;
3. Trypsin and Indole-Amidine;

these are systems from the published paper ["Interactive molecular dynamics in virtual  reality for accurate flexible protein-ligand docking"](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0228461), further details can be found in the `system_descriptions.md` file in `openmm_files` directory, where the files are stored. <br>

As we are going to run the simulations using OpenMM, we need to create an `OpenMMSimulation` for each one. <br>
In the same cell, we also define the visualisation rendering for each system: by identifying the atoms belonging to the protein and the ligand (e.g.: in .xml file), we set the protein to be visualised as 'cartoon' and the ligand as 'liquorice'. With this selection method, the chosen complex automatically displays the specified rendering each time it is played (see ["Setting default visualisations for simulations in NanoVer's OmniRunner"](https://github.com/IRL2/nanover-protocol/blob/main/examples/omni_multiple.ipynb) for another example).

In [2]:
# Loading simulation file
neuraminidase_zanamivir = "openmm_files/6hcx_complex.xml"
nz_simulation = OpenMMSimulation.from_xml_path(neuraminidase_zanamivir)

# Defining selections and rendering for protein and ligand
neuraminidase_selection = RenderingSelection("selection.protein")
neuraminidase_selection.set_particles(list(range(0,5966)))
neuraminidase_selection.renderer =  'cartoon'

zanamivir_selection = RenderingSelection("selection.ligand")
zanamivir_selection.set_particles(list(range(5966, 6010)))
zanamivir_selection.renderer = {
    'color': {
        'type': 'cpk',   # we are defining the color scheme, cpk, using the built-in NanoVer version
        'scheme': 'nanover'
    },
    'scale': 0.04,      # the scale (size) 
    'render': 'liquorice'
}

In [3]:
# Loading simulation file
hiv_amprenavir = "openmm_files/hiv1_complex.xml"
ha_simulation = OpenMMSimulation.from_xml_path(hiv_amprenavir)

# Defining selections and rendering for protein and ligand
hiv_selection = RenderingSelection("selection.protein")
hiv_selection.set_particles(list(range(0,3129)))
hiv_selection.renderer =  'cartoon'

amprenavir_selection = RenderingSelection("selection.ligand")
amprenavir_selection.set_particles(list(range(3129, 3200)))
amprenavir_selection.renderer = {
    'color': {
        'type': 'cpk',
        'scheme': 'nanover'
    },
    'scale': 0.04,
    'render': 'liquorice'
}

In [4]:
# Loading simulation file
trypsin_indoleamidine = "openmm_files/2g5n_complex.xml"
ti_simulation = OpenMMSimulation.from_xml_path(trypsin_indoleamidine)

# Defining selections and rendering for protein and ligand
trypsin_selection = RenderingSelection("selection.protein")
trypsin_selection.set_particles(list(range(0,3220)))
trypsin_selection.renderer =  'cartoon'

indoleamidine_selection = RenderingSelection("selection.ligand")
indoleamidine_selection.set_particles(list(range(3220, 3256)))
indoleamidine_selection.renderer = {
    'color': {
        'type': 'cpk',
        'scheme': 'nanover'
    },
    'scale': 0.04,
    'render': 'liquorice'
}

Now we can create an `OmniRunner` to serve and load the Openmm simulations created above onto it.

In [5]:
imd_runner = OmniRunner.with_basic_server(nz_simulation, ha_simulation, ti_simulation, name="my-nanover-server")

# within the OmniRunner we set the specific selections for each simulation
imd_runner.set_simulation_selections(nz_simulation, neuraminidase_selection, zanamivir_selection)
imd_runner.set_simulation_selections(ha_simulation, hiv_selection, amprenavir_selection)
imd_runner.set_simulation_selections(ti_simulation, trypsin_selection, indoleamidine_selection)

In [6]:
imd_runner.print_basic_info()

Serving "my-nanover-server" (ws://localhost:38801), discoverable on all interfaces on port 54545
Available simulations:
0: "6hcx_complex"
1: "hiv1_complex"
2: "2g5n_complex"


All set up! Now run the cell below to start the first simulation (neuraminidase and zanamivir complex) !

In [7]:
# Start the first simulation
imd_runner.load(0)
imd_runner.play()

## Jupyter notebook interface

Through the notebook you can switch among the simulations in two ways. <br>
Using the `.next()` method, one can sequentially pass through the simulations loaded onto the server according to the order you set the `OmniRunner.with_basic_server(...)`. <br> 



In [8]:
# Switch to the second simulation
imd_runner.next() 

In this case, after running the cell above, you will visualise the HIV and amprenavir complex. <br>
Once you have finished with the second simulation, run the cell below to start the third simulation to play with the trypsin and indoleamidine system.

In [9]:
# Switch to the third simulation
imd_runner.next()

It is also possible to switch to a specific simulation with the following method:

In [10]:
imd_runner.load(0) # Select which simulation you want to load (0 , 1 , 2 )
imd_runner.play() # play simulation

Using `.load()` you can choose the one you prefer and then `.play` it. Each simulation has a number assigned, starting from `0`, according to the order you set the `OmniRunner.with_basic_server(...)`. <br>

### Demo controls

In [11]:
from nanover.jupyter import show_runner_controls

show_runner_controls(imd_runner)

interactive(children=(Dropdown(description='Force Type', index=1, options=(('Gaussian', 'gaussian'), ('Harmoni…

interactive(children=(IntSlider(value=100, description='Force Scale', max=1000, min=1), Output()), _dom_classe…

interactive(children=(FloatSlider(value=0.4, description='Force Range', max=2.0, min=0.1, step=0.05), Output()…

interactive(children=(FloatSlider(value=1.0, description='Passthrough', max=1.0), Output()), _dom_classes=('wi…

### NGLView visualisation

In [12]:
from nanover.nglview import NGLClient

client = NGLClient.from_runner(imd_runner)
client.view

NGLWidget()

## VR interface 

There is the possibility to switch among simulation files directly while in VR space. <br>
You need to access the menu holding the left joystick forward, select 'menu' then 'sims', which contains all the simulations loaded.
Choose the one you prefer and it will be loaded inside NanoVer. You can repeat this action how many times you prefer. <br>
Here there is a gif illustrating the process:

![SegmentLocal](images/MultipleFiles.gif "segment")

## Close the server
Well done! You've learned how to run multiple simulation files on a single server and set default visualization for each.
When you have finished with your session, close the server running the cell below. 

In [13]:
imd_runner.close()

## Next Step

1. Learn how to set up an OpenMM simulation from scratch to generate an .xml file compatible with NanoVer [here](../openmm/openmm_neuraminidase.ipynb).
2. Refer to this [notebook](../fundamentals/visualisations.ipynb) to explore additional visualization methods and rendering options.